# Biotrainer Introduction Notebook: Compare results

In the following tutorial, we will use [biotrainer](https://github.com/sacdallago/biotrainer) to train a baseline model (negative control) for an existing predictor. We will use one_hot_encodings for that. Can we claim that our predictor performs better than the baseline?

## 0. Install Biotrainer

In [ ]:
# Create a virtual environment before, then run:
!pip install biotrainer

## 1. The prediction task and predictor

In our example, we already trained a regression model on the [FLIP meltome dataset split mixed](https://github.com/J-SNACKKB/FLIP/tree/main/splits/meltome) with [ProtT5](https://github.com/agemagician/ProtTrans) embeddings. So, we created a model that predicts the meltdown temperature for a given protein sequence.

A common metric to compare performance of regression models is the `root_mean_squared_error` (rmse). The "CI" variable means "confidence interval". Both the mean for the rmse and the ci come from bootstrapping the test set with the created model. This is to draw repeatedly from the test set with replacement (e.g. 30 times) and then calculate the performance on that set. You can learn more about bootstrapping here: https://math.mit.edu/~dav/05.dir/class24-prep-a.pdf 

Now, we already have the result on the test set for ProtT5 given:

In [ ]:
from biotrainer.protocols import Protocol

task_protocol = Protocol.sequence_to_value
prott5_rmse_mean = 9.564  # Not real data
prott5_rmse_ci = 0.63  # Not real data

# Bootstrapping config (sample_size == length of test set)
bootstrapping_confidence_level = 0.05
bootstrapping_iterations = 30

## 2. Downloading the data

For this regression task, we can download one simple FASTA file that contains both our sequences and the respective targets (meltdown temperature of the sequence).

In [ ]:
import requests

url = "https://nextcloud.cit.tum.de/index.php/s/PY2bpHL8wJxJMHq/download"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

file_name = "biotrainer_meltome_mixed.fasta"
response = requests.get(url, headers=headers)
with open(file_name, "wb") as f:
    f.write(response.content)

## 3. Defining the biotrainer configuration

We already know our sequences and targets are stored in the downloaded fasta file. We also know the protocol and which embedder to use. The rest of the configurations are straight-forward.

In [ ]:
biotrainer_config = {
    "protocol": task_protocol.name,  # We have a per-sequence regression task, so sequence_to_value
    "input_file": file_name,  # The path to the downloaded fasta file
    "model_choice": "FNN",  # Fully connected neural network
    "embedder_name": "one_hot_encoding",  # Name of the embedding model
    "num_epochs": 200,  # Define how long we want to train at max
}

## 4. Running the experiment

We are now ready to train our baseline prediction model!

In [ ]:
from biotrainer.utilities.cli import train

results = train(biotrainer_config)

## 5. Comparing the results

After our baseline model finished training, we can now compare it to the ProtT5 result.

By the way, did you notice that the model did not train for the given 200 epochs? That is because there is a built-in mechanism called `early stopping` which should be used for any deep learning model. It avoids overfitting by stopping the training after a certain amount of epochs that did not increase the performance on the validation set. This "certain amount" can be defined in the training configuration by using `"patience": 50 (or some other positive integer value)`.  

In [ ]:
# First we get the bootstrapping results
bootstrapping_ohe_dict = results["test_results"]["test"]["bootstrapping"]
ohe_rmse_mean = bootstrapping_ohe_dict["rmse"]["mean"]
ohe_rmse_ci = (bootstrapping_ohe_dict["rmse"]["upper"] - bootstrapping_ohe_dict["rmse"]["lower"]) / 2  # Assuming about symmetrical bounds

# Now let's compare them!
print(f"ProtT5: RMSE Mean: {prott5_rmse_mean} CI: {prott5_rmse_ci}")
print(f"OHE: RMSE Mean: {ohe_rmse_mean} CI: {ohe_rmse_ci}")

## Final step: z-test

Okay, only from looking at this, we can already calculate in our heads, that the confidence intervals do not overlap. That means that there is a significant difference between the two test set performances in our setup.

But let's first verify that by using a statistical test, the so called "Z-Test". 
The Z-test essentially asks: 'Given these RMSE scores and their uncertainties from bootstrapping, what's the probability that the difference between ProtT5 and the simpler approach occurred by chance?'

It works by:

* Measuring how far apart the two RMSE means are
* Taking into account how uncertain each measurement is (using the confidence intervals from our bootstrapping)
* Comparing this to what we'd expect if there was no real difference between the approaches

For example, if ProtT5 gives an RMSE of 3.2±0.2 and the simpler approach gives 4.1±0.2, the Z-test helps us confirm that ProtT5's better performance is 'real' and not just random variation. The test gives us a p-value: if it's less than 0.05, we can say that one approach is significantly better than the other.

In [ ]:
import numpy as np
from scipy import stats

def z_test(mean1, ci1, mean2, ci2):
    # Convert CI to standard error for 95% CI
    se1 = ci1 / 1.96
    se2 = ci2 / 1.96
    z_stat = (mean1 - mean2) / np.sqrt(se1**2 + se2**2)
    p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # two-tailed
    return z_stat, p_value

z_stat, p_value = z_test(prott5_rmse_mean, prott5_rmse_ci, ohe_rmse_mean, ohe_rmse_ci)
significant = p_value < 0.05

print(z_stat, p_value)
print(f"As {p_value} {'<' if significant else '>='} 0.05, we can assume that there is {'a' if significant else 'no'} significant difference between the performance of ProtT5 and one hot encodings!")